In [1]:
import requests
from bs4 import BeautifulSoup
import re
import ast

base_url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='
base_iframe_url = 'https://movie.naver.com'
# https://movie.naver.com/movie/bi/mi/point.nhn?code=18906 -> 리뷰없음
# https://movie.naver.com/movie/bi/mi/point.nhn?code=60503 -> 영화없음

In [18]:
crawling_data = []
for code in range(69900,70100):
    movie_url = base_url + str(code)
    req = requests.get(movie_url)
    print(movie_url)
    
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    iframe = soup.find_all('iframe')
    
    # 영화가 존재하지 않는 경우 체크
    if not iframe:
        continue
    
    iframe_url = base_iframe_url + iframe[0]['src']
    req = requests.get(iframe_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    score_table = soup.find(name="div", attrs={"class":"score_result"})
    
    # 리뷰가 존재하지 않는 경우 체크
    if not score_table:
        continue
    
    score_list = score_table.find_all(name="div", attrs={"class":"star_score"})
    reple_list = score_table.find_all(name="div", attrs={"class":"score_reple"})
    
    for idx in range(0, len(score_list)):
        if reple_list[idx].find('p') and score_list[idx].find('em'):
            reple = reple_list[idx].find('p').text
            score = score_list[idx].find('em').text
            if int(score) < 6:
                score = '0'
            else:
                score = '1'
            crawling_data.append([reple, score])

https://movie.naver.com/movie/bi/mi/point.nhn?code=69900
https://movie.naver.com/movie/bi/mi/point.nhn?code=69901
https://movie.naver.com/movie/bi/mi/point.nhn?code=69902
https://movie.naver.com/movie/bi/mi/point.nhn?code=69903
https://movie.naver.com/movie/bi/mi/point.nhn?code=69904
https://movie.naver.com/movie/bi/mi/point.nhn?code=69905
https://movie.naver.com/movie/bi/mi/point.nhn?code=69906
https://movie.naver.com/movie/bi/mi/point.nhn?code=69907
https://movie.naver.com/movie/bi/mi/point.nhn?code=69908
https://movie.naver.com/movie/bi/mi/point.nhn?code=69909
https://movie.naver.com/movie/bi/mi/point.nhn?code=69910
https://movie.naver.com/movie/bi/mi/point.nhn?code=69911
https://movie.naver.com/movie/bi/mi/point.nhn?code=69912
https://movie.naver.com/movie/bi/mi/point.nhn?code=69913
https://movie.naver.com/movie/bi/mi/point.nhn?code=69914
https://movie.naver.com/movie/bi/mi/point.nhn?code=69915
https://movie.naver.com/movie/bi/mi/point.nhn?code=69916
https://movie.naver.com/movie/b

In [20]:
crawling_data[5]

['진짜 재밋내..굿 강추  ', '1']

In [34]:
import csv

with open('review_data.csv', 'w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Review', 'Like'])
    for idx in range(0, len(crawling_data)):
        review = crawling_data[idx][0]
        like = crawling_data[idx][1]
        writer.writerow([review, like])

In [40]:
import pandas as pd

df = pd.read_csv('review_data.csv')
df.tail(5)

,Review,Like
741,사랑과 열정에 국경이 '있음'을 문화적 차이로 리얼하게 보여준 사랑가득한 영화.,1
742,왜 로카르노 수상작인지... ㅡㅡ;;;,1
743,"줄거리를 피프에서 배껴온듯한데 본 영화정보완 다르네요, 아무튼 재밌게봤습니다",1
744,망한다에 200만표 건다,0
745,찍기귀찮으셨나? 무슨다 롱테이크야.. 대사는많고,0


Review Classification (TF)

In [55]:
import os

# 리뷰데이터 다운로드 - https://github.com/e9t/nsmc
train_data = pd.read_csv('rating_train.csv')
test_data = pd.read_csv('review_data.csv')
train_data = train_data.dropna()
test_data = test_data.dropna()

In [71]:
# 사전 생성을 위한 형태소 추출
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

NameError: name 'jpype' is not defined

In [70]:
# (형태소 뭉치, label) 을 element로 하는 list 생성
train_docs = [(tokenize(row['review']), row['like']) for index, row in train_data.iterrows()]
test_docs = [(tokenize(row['review']), row['like']) for index, row in test_data.iterrows()]

NameError: name 'tokenize' is not defined

In [69]:
# 생성된 리스트에서 말뭉치 추출
token = [t for d in train_docs for t in d[0]]

# 자연어 처리 모듈 nltk로, 말뭉치에서 빈도수 상위 n개 단어 추출
import nltk
text = nltk.Text(token, name="NMSC") # text.vocab().most_common(n)

NameError: name 'train_docs' is not defined

In [ ]:
# 상위 2000개 빈도수의 단어를 추출한 뒤, BoW(Bag of Words)기법으로 워드임베딩
selected_words = [f[0] for f in text.vocab().most_common(2000)]

In [ ]:
# nltk 라이브러리가 원하는 형태로 input formatting
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

train_docs = train_docs[:10000]
test_docs = test_docs[:10000]
train_xy = [(term_exists(d), c) for d,c in train_docs]
test_xy = [(term_exists(d), c) for d,c in test_docs]

In [60]:
# 약 80%의 정확도
classifier = nltk.NaiveBayesClassifier.train(train_xy)
print(nltk.classify.accuracy(classifier, test_xy))

NameError: name 'nltk' is not defined

TF-IDF Classification

In [58]:
import os
import numpy as np

# 리뷰데이터
train_data = pd.read_csv('rating_train.csv')
test_data = pd.read_csv('review_data.csv')
train_data = train_data.dropna()
test_data = test_data.dropna()

# tf-idf 임베딩
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

vect = TfidfVectorizer()
review_column = np.array([row['review'] for index, row in train_data.iterrows()])
y_train = np.array([row['like'] for index, row in train_data.iterrows()])

# fit_transform : vect object에게 transform을 학습시킴
X_train = vect.fit_transform(review_column)

# 로지스틱 분류기 학습 : 약 81% 정확도
from sklearn.linear_model import LogisticRegression
lrm = LogisticRegression()
lrm.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [66]:
review_column = np.array([row['Review'] for index, row in test_data.iterrows()])
y_test = np.array([row['Like'] for index, row in test_data.iterrows()])

# transform : test 데이터를 학습된 vect object로 임베딩시킴
X_test = vect.transform(review_column)
lrm.score(X_test, y_test)

0.7265415549597856

In [68]:
# 실제 텍스트로 테스트
test_data1 = ["그저그럼.. 재미없음"] # label : 0
test_data2 = ["꿀잼 최고의 영화"] # label : 1
test_tfidf1 = vect.transform(test_data1)
test_tfidf2 = vect.transform(test_data2)
print(lrm.predict(test_tfidf1))
print(lrm.predict(test_tfidf2))

[0]
[1]


In [65]:
# 응용 어플리케이션 임베딩을 위한 모델 관련 파일 저장
import pickle
import os

pickle.dump(vect, open(os.path.join('vectorizer.pkl'), 'wb'))
vect_read = pickle.load(open(os.path.join('vectorizer.pkl'), 'rb'))

pickle.dump(lrm, open(os.path.join('classification_model.pkl'), 'wb'))
lrm_read = pickle.load(open(os.path.join('classification_model.pkl'), 'rb'))